In [2]:
import pandas as pd
from pathlib import Path

# ========== CONFIG ==========
CSV_PATH = Path("/Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/players_dom_metrics_filled.csv")

PLAYER_NAME = "Joe Burrow"   # <-- change this
COLUMN_NAME = "BTT%3"         # <-- change this (any column you want to inspect)

# ========== LOAD ==========
df = pd.read_csv(CSV_PATH)

if "player_name" not in df.columns:
    raise ValueError("Expected a 'player_name' column in the CSV.")

if COLUMN_NAME not in df.columns:
    raise ValueError(f"Column '{COLUMN_NAME}' not found in CSV.")

# ========== FILTER PLAYER ==========
player_row = df[df["player_name"].str.contains(PLAYER_NAME, case=False, na=False)]

if player_row.empty:
    print(f"❌ Player not found: {PLAYER_NAME}")
else:
    value = player_row.iloc[0][COLUMN_NAME]
    
    print(f"✅ Player found: {player_row.iloc[0]['player_name']}")
    print(f"🔍 Column checked: {COLUMN_NAME}")
    print(f"📊 Value: {value}")

    if pd.isna(value):
        print("⚠️ This value is NaN (missing).")
    else:
        print("✅ This value is present (filled).")

    # Optional: show the full row for context
    display(player_row)


✅ Player found: Joe Burrow
🔍 Column checked: BTT%3
📊 Value: nan
⚠️ This value is NaN (missing).


,player_name,pos,team,conf,draft_year,draft_round,pick_in_round,pick_overall,pfr_draft_url,Year1,...,BTT%5,TWP%5,DAA5,YPA5,MTF5,FUM5,10+5,REC%5,Y/REC5,DRP%5
1657,Joe Burrow,QB,LSU,SEC,2020,1,1,1,https://www.pro-football-reference.com/years/2...,2016.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
